In [1]:
"""
SEE: https://github.com/sudoswap/lssvm/blob/main/src/LSSVMPair.sol
Events to query for are:
  1. SwapNFTInPair()
    - emitted by LSSVMPair.swapNFTsForToken()  (sell order)
    - has no data so need to look at Transfer event for volume
  2. SwapNFTOutPair()
    - emitted by LSSVMPair.swapTokenForAnyNFTs() (buy order)
    - has no data so need to look at Transfer event for volume
  3. SpotPriceUpdate(uint128 newSpotPrice)
    - emitted by LSSVMPair on spot price changes: use to track price changes
"""
# switch to sudo-ape wd first for ape project to work properly
import os
from pathlib import Path

if Path(*Path(os.getcwd()).parts[-2:]) == Path('sudo-ape/notebook'):
    os.chdir('..')

In [3]:
import pandas as pd
import typing as tp

from ape import accounts, chain, Contract, networks, project

In [4]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [5]:
current_block = chain.blocks[-1].number
current_block

15466430

In [6]:
sudo_router = Contract("0x2B2e8cDA09bBA9660dCA5cB6233787738Ad68329")
sudo_router

<LSSVMRouter 0x2B2e8cDA09bBA9660dCA5cB6233787738Ad68329>

In [7]:
sudo_factory = Contract("0xb16c1342E617A5B6E4b631EB114483FDB289c0A4")
sudo_factory

<LSSVMPairFactory 0xb16c1342E617A5B6E4b631EB114483FDB289c0A4>

In [8]:
sudo_factory.NewPair

NewPair(address poolAddress)

In [9]:
# use address of top xmons pool to check is pool thru factory
sudo_factory.isPair("0x5caf332dca4e6c9e69d52f320c21e74845353db0", 0)

True

In [10]:
sudo_pair_xmons = Contract("0x5caf332dca4e6c9e69d52f320c21e74845353db0")
sudo_pair_xmons

<LSSVMPair 0x5caF332DcA4e6C9E69d52F320c21e74845353db0>

In [11]:
# utility functions to unfold event arguments into separate columns
def unfold_event_args(key: str, event_args: dict) -> tp.Any:
    return event_args[key]

def unfold(df: pd.DataFrame):
    for key in df.iloc[-1]['event_arguments'].keys():
        df[key] = df['event_arguments'].apply(lambda ev: unfold_event_args(key, ev))

In [12]:
%time query_pairs = sudo_factory.NewPair.query("*", start_block=current_block - 3000)  # last 10 hr
query_pairs

INFO: Cache database has not been initialized
CPU times: user 309 ms, sys: 40.5 ms, total: 349 ms
Wall time: 2.19 s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x5b47cD64e53238dD69a2C08Aadd...,0x25ba625a3a7d8e5b2c3bfa2c5875e5590bbb855a4007...,15463467,0xff083983c3098b35929879d3c71f6729f5f0d5a626c7...,91,24
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x84b36CAd308A3F2F9EFcC0780DB...,0x23db577bad819d82ded810ed3cff49240cc8a5a7db30...,15463503,0xc4998bf62c48bb9430070b49ca2f21029d3e006ceb01...,133,48
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x9D97Ec0bfFd47eebbB4cCA5554c...,0x1201541eba18b42f4351f814dd2045ea6e56cdc28586...,15463575,0xe4609205669ab7fb2b3779904aa7c9b0cb03b3bf9402...,32,11
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xbd2A2Bb05e9CA3c38E998Db8b49...,0x374913ea025c532d34663c60079273a678ae447d3618...,15463580,0xc7c06057d97b5f18bfc4396f2616c6177c213e4e8316...,136,96
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xA274944181784ccf498687eaAb0...,0x86771eee47cdfacda1fffdeea5bc801821874b3d6485...,15463589,0x2f7dcdccf3975d8219bdfdcc95478ef6cc1868b37d24...,160,46
...,...,...,...,...,...,...,...,...
133,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x764cB6De1d3b63F073796Bb2273...,0x87ec7d29f0cb1120004c5dcc61ec5042d24703f15d95...,15466386,0x9583664811ca295daf56f9cb42a19482a1847be9494b...,131,106
134,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x3AB1B23dC6b808Ff2f7eb8B2327...,0xe0555473b01fcf1d8843140e89a5fdab5e677068b610...,15466386,0x9583664811ca295daf56f9cb42a19482a1847be9494b...,135,107
135,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x34a8257CC85030085B9AFf1a455...,0x20d13af3edd10c07ec4a10515e22919ed16076d8986e...,15466390,0xe730d3978e6816209210707ddfa3808e0e0864be8c71...,109,62
136,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x66fbc8FBe714dB920136f40AccD...,0x197673083b0b680f3eac7115781746c3e6e7abfe1019...,15466401,0x91583caa46b1e61a8a777354f00dfd113d27041d67c1...,89,65


In [13]:
unfold(query_pairs)

In [14]:
query_pairs

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,poolAddress
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x5b47cD64e53238dD69a2C08Aadd...,0x25ba625a3a7d8e5b2c3bfa2c5875e5590bbb855a4007...,15463467,0xff083983c3098b35929879d3c71f6729f5f0d5a626c7...,91,24,0x5b47cD64e53238dD69a2C08AaddCc5fB04539f71
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x84b36CAd308A3F2F9EFcC0780DB...,0x23db577bad819d82ded810ed3cff49240cc8a5a7db30...,15463503,0xc4998bf62c48bb9430070b49ca2f21029d3e006ceb01...,133,48,0x84b36CAd308A3F2F9EFcC0780DBEB9286E25Da0a
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x9D97Ec0bfFd47eebbB4cCA5554c...,0x1201541eba18b42f4351f814dd2045ea6e56cdc28586...,15463575,0xe4609205669ab7fb2b3779904aa7c9b0cb03b3bf9402...,32,11,0x9D97Ec0bfFd47eebbB4cCA5554cf017CF1b96269
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xbd2A2Bb05e9CA3c38E998Db8b49...,0x374913ea025c532d34663c60079273a678ae447d3618...,15463580,0xc7c06057d97b5f18bfc4396f2616c6177c213e4e8316...,136,96,0xbd2A2Bb05e9CA3c38E998Db8b497C695b4D8c3eA
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xA274944181784ccf498687eaAb0...,0x86771eee47cdfacda1fffdeea5bc801821874b3d6485...,15463589,0x2f7dcdccf3975d8219bdfdcc95478ef6cc1868b37d24...,160,46,0xA274944181784ccf498687eaAb008b78698d595d
...,...,...,...,...,...,...,...,...,...
133,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x764cB6De1d3b63F073796Bb2273...,0x87ec7d29f0cb1120004c5dcc61ec5042d24703f15d95...,15466386,0x9583664811ca295daf56f9cb42a19482a1847be9494b...,131,106,0x764cB6De1d3b63F073796Bb227359284a237c911
134,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x3AB1B23dC6b808Ff2f7eb8B2327...,0xe0555473b01fcf1d8843140e89a5fdab5e677068b610...,15466386,0x9583664811ca295daf56f9cb42a19482a1847be9494b...,135,107,0x3AB1B23dC6b808Ff2f7eb8B23279cF969FC70a63
135,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x34a8257CC85030085B9AFf1a455...,0x20d13af3edd10c07ec4a10515e22919ed16076d8986e...,15466390,0xe730d3978e6816209210707ddfa3808e0e0864be8c71...,109,62,0x34a8257CC85030085B9AFf1a4556F1247f273689
136,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x66fbc8FBe714dB920136f40AccD...,0x197673083b0b680f3eac7115781746c3e6e7abfe1019...,15466401,0x91583caa46b1e61a8a777354f00dfd113d27041d67c1...,89,65,0x66fbc8FBe714dB920136f40AccDF669e699FE648


In [15]:
sudo_pair_xmons.SpotPriceUpdate

SpotPriceUpdate(uint128 newSpotPrice)

In [16]:
%time query_prices_xmons = sudo_pair_xmons.SpotPriceUpdate.query('*', start_block=current_block - 10000)
query_prices_xmons

CPU times: user 463 ms, sys: 64.3 ms, total: 527 ms
Wall time: 4.41 s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 19830097087378640000},0x0b8bbb0a775e0d41b1a761df0af456a2684147d15e9b...,15456511,0xc775c2fb5902054be8abe871097dcf95061990eecfed...,11,13
1,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0xfe1a9b9f022419928d1b6db5d32a9b16189626aa63bc...,15456556,0xe9e268f542ed42dfd8d8c539254f5b4651907fd2b9ae...,290,31
2,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 19830097087378640000},0x7f1c135dce08e68fdcbc8bc4d14fb5e7c61d9a18b763...,15456751,0xf14069bbcb95296f9777c5325636fc1a236032e81bb0...,180,106
3,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 24830097087378640000},0x71a7117a7cf2c88ee9a7391e8beb5ffdf48b6fad80b6...,15456797,0x31416626ce15fcd73a0ed04b773e65f0e0e3f00db735...,126,92
4,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 22330097087378640000},0x7c5503141c6f5e732d48a2f9685d3f33982d7afa226f...,15456811,0x9f34d19c6d84f498e9acf51fbea4f4c638a6d66a64c1...,4,0
5,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 19830097087378640000},0x9373336e605da905261f7821284e1a1703657ab25297...,15456866,0x0759193357cef7d05121d9ace57add71a35b39d158d3...,13,9
6,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0x0fd31f0b13eb8715d224612e60725b9d85afa829743f...,15456888,0x13193f4c7b33949a000ac2b287618fb7cb4f220f6fd9...,125,51
7,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 19830097087378640000},0x4e47bb19abe6413b9b3368650205a3f3bfca25466c98...,15456916,0xcd469de70c8bd991ff90a8801c55af53d80075357654...,15,19
8,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0x7fb93d294a35eb0e184ae83d169483349a1a6eada338...,15456946,0x638a76b25b914620bd5b862225d3ffac7387389f7e5e...,163,91
9,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 14830097087378640000},0xb195b35e2248ac0968704271b99b747a78fcebc13c2f...,15458203,0xae46c543e4df2363b50176c30b301cff6d451a15a843...,26,27


In [17]:
unfold(query_prices_xmons)

In [18]:
query_prices_xmons

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,newSpotPrice
0,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 19830097087378640000},0x0b8bbb0a775e0d41b1a761df0af456a2684147d15e9b...,15456511,0xc775c2fb5902054be8abe871097dcf95061990eecfed...,11,13,19830097087378640000
1,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0xfe1a9b9f022419928d1b6db5d32a9b16189626aa63bc...,15456556,0xe9e268f542ed42dfd8d8c539254f5b4651907fd2b9ae...,290,31,17330097087378640000
2,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 19830097087378640000},0x7f1c135dce08e68fdcbc8bc4d14fb5e7c61d9a18b763...,15456751,0xf14069bbcb95296f9777c5325636fc1a236032e81bb0...,180,106,19830097087378640000
3,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 24830097087378640000},0x71a7117a7cf2c88ee9a7391e8beb5ffdf48b6fad80b6...,15456797,0x31416626ce15fcd73a0ed04b773e65f0e0e3f00db735...,126,92,24830097087378640000
4,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 22330097087378640000},0x7c5503141c6f5e732d48a2f9685d3f33982d7afa226f...,15456811,0x9f34d19c6d84f498e9acf51fbea4f4c638a6d66a64c1...,4,0,22330097087378640000
5,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 19830097087378640000},0x9373336e605da905261f7821284e1a1703657ab25297...,15456866,0x0759193357cef7d05121d9ace57add71a35b39d158d3...,13,9,19830097087378640000
6,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0x0fd31f0b13eb8715d224612e60725b9d85afa829743f...,15456888,0x13193f4c7b33949a000ac2b287618fb7cb4f220f6fd9...,125,51,17330097087378640000
7,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 19830097087378640000},0x4e47bb19abe6413b9b3368650205a3f3bfca25466c98...,15456916,0xcd469de70c8bd991ff90a8801c55af53d80075357654...,15,19,19830097087378640000
8,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 17330097087378640000},0x7fb93d294a35eb0e184ae83d169483349a1a6eada338...,15456946,0x638a76b25b914620bd5b862225d3ffac7387389f7e5e...,163,91,17330097087378640000
9,SpotPriceUpdate,0x5caF332DcA4e6C9E69d52F320c21e74845353db0,{'newSpotPrice': 14830097087378640000},0xb195b35e2248ac0968704271b99b747a78fcebc13c2f...,15458203,0xae46c543e4df2363b50176c30b301cff6d451a15a843...,26,27,14830097087378640000


In [27]:
# local contract container works :)
pair = project.LSSVMPair.at(query_pairs['poolAddress'].iloc[57])
pair

<LSSVMPair 0x401c6d66C36A35ECdF557489170B83328b924214>

In [28]:
query_pairs['poolAddress'].iloc[57]

'0x401c6d66C36A35ECdF557489170B83328b924214'

In [29]:
pair_nft_address = pair.nft()
pair_nft_address

'0x65784d6F23DE30A17122E96c0F0986C378ed6947'

In [31]:
# checks out with etherscan
Contract(pair_nft_address)

<BoredApeYogaClub 0x65784d6F23DE30A17122E96c0F0986C378ed6947>